In [ ]:
pip install transformers>=4.4.0

In [ ]:
from transformers import BertTokenizer, BertModel, BartForConditionalGeneration, BartTokenizer
import torch
import pandas as pd
import csv

def read_csv_to_dataframe(filename):
    df = pd.read_csv(filename)
    return df

def append_sentence_to_csv(sentence, filename):
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([sentence])

def summarization(inp):
  # Define the input text
  input_text = inp

  # Tokenize the input text with BERT tokenizer
  tokens = bert_tokenizer.encode_plus(input_text, truncation=True, max_length=512, padding='longest', return_tensors='pt')

  # Extract BERT features
  with torch.no_grad():
    bert_outputs = bert_model(**tokens)

  # Generate the summary using BART
  bart_inputs = bart_tokenizer.prepare_seq2seq_batch([input_text], truncation=True, max_length=1024, padding='longest', return_tensors='pt')
  bart_inputs = bart_inputs.to(bart_model.device)

  summary_ids = bart_model.generate(
    input_ids=bart_inputs.input_ids,
    attention_mask=bart_inputs.attention_mask,
    max_length=100,
    min_length=30,
    num_beams=4,
    length_penalty=2.0,
    no_repeat_ngram_size=3,
    early_stopping=True
  )

  # Decode the summary tokens
  summary = bart_tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

  # Print the summary
  #print("Summary:", summary)
  return (summary)

if __name__ == "__main__":
  csv_filename = "/content/English1 - Sheet1.csv"
  dataframe = read_csv_to_dataframe(csv_filename)
  article = pd.DataFrame(dataframe, columns=["article"])
  #print(article)
  # Load the BERT model and tokenizer
  bert_model_name = 'bert-base-uncased'
  bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
  bert_model = BertModel.from_pretrained(bert_model_name)

  # Load the BART model and tokenizer
  bart_model_name = 'facebook/bart-base'
  bart_tokenizer = BartTokenizer.from_pretrained(bart_model_name)
  bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name)

  result_df = pd.DataFrame(columns=["Summary"])
  csv_filename = "summ2.csv"

  for index in range(len(dataframe)):
    text=dataframe.loc[index, 'article']
    summ = summarization(text)
    append_sentence_to_csv(summ,csv_filename)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
pip install rouge

In [ ]:
import csv
import pandas as pd
from rouge import Rouge

def read_csv_to_dataframe(filename):
    df = pd.read_csv(filename)
    return df

if __name__ == "__main__":
  csv_1 = "/content/English1 - Sheet1.csv"
  act_df = read_csv_to_dataframe(csv_1)
  act_sum = pd.DataFrame(act_df)
  act_su = act_sum['highlights'].tolist()
  print(type(act_su))

  csv_2 = "/content/summ2 - summ2.csv.csv"
  gen_df = read_csv_to_dataframe(csv_2)
  gen_sum = pd.DataFrame(gen_df)
  gen_su = gen_sum['summary'].tolist()
  print(type(gen_su))

  rouge = Rouge()
  scores = rouge.get_scores(gen_su,act_su)

  rouge_1_f_score = scores[0]["rouge-1"]["f"]
  rouge_2_f_score = scores[0]["rouge-2"]["f"]
  rouge_l_f_score = scores[0]["rouge-l"]["f"]

  print("ROUGE-1 F-score:", rouge_1_f_score)
  print("ROUGE-2 F-score:", rouge_2_f_score)
  print("ROUGE-L F-score:", rouge_l_f_score)


<class 'list'>
<class 'list'>
ROUGE-1 F-score: 0.3364485938649664
ROUGE-2 F-score: 0.15517240972205718
ROUGE-L F-score: 0.3177570050799197


In [ ]:
rouge_1_f1 = scores[0]['rouge-1']['f']
rouge_1_recall = scores[0]['rouge-1']['r']
rouge_1_precision = scores[0]['rouge-1']['p']

print("ROUGE-1 Metric:")
print(rouge_1_f1)
print(rouge_1_recall)
print(rouge_1_precision)

rouge_2_f1 = scores[0]['rouge-2']['f']
rouge_2_recall = scores[0]['rouge-2']['r']
rouge_2_precision = scores[0]['rouge-2']['p']

print("ROUGE-2 Metric:")
print(rouge_2_f1)
print(rouge_2_recall)
print(rouge_2_precision)

rouge_l_f1 = scores[0]['rouge-l']['f']
rouge_l_recall = scores[0]['rouge-l']['r']
rouge_l_precision = scores[0]['rouge-l']['p']

print("ROUGE-l Metric:")
print(rouge_l_f1)
print(rouge_l_recall)
print(rouge_l_precision)

ROUGE-1 Metric:
0.3364485938649664
0.5454545454545454
0.24324324324324326
ROUGE-2 Metric:
0.15517240972205718
0.2727272727272727
0.10843373493975904
ROUGE-l Metric:
0.3177570050799197
0.5151515151515151
0.22972972972972974


In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BartForConditionalGeneration, BartTokenizer
import pandas as pd
import csv

def read_csv_to_dataframe(filename):
    df = pd.read_csv(filename)
    return df

def append_sentence_to_csv(sentence, filename):
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([sentence])

def summarization(inp):
    # Define the input text
    input_text = inp

    # Tokenize the input text with BERT tokenizer
    tokens = bert_tokenizer(input_text, truncation=True, max_length=400, padding='max_length', return_tensors='pt')

    # Extract BERT features
    with torch.no_grad():
        bert_outputs = bert_model(**tokens)

    # Generate the summary using BART
    bart_inputs = bart_tokenizer.prepare_seq2seq_batch([input_text], truncation=True, max_length=900, padding='max_length', return_tensors='pt')

    summary_ids = bart_model.generate(
        input_ids=bart_inputs.input_ids.to(bart_model.device),
        attention_mask=bart_inputs.attention_mask.to(bart_model.device),
        max_length=400,
        min_length=50,
        num_beams=6,
        length_penalty=3.0,
        no_repeat_ngram_size=4,
        early_stopping=True
    )

    # Decode the summary tokens
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

if __name__ == "__main__":
    csv_filename = "/content/English1 - Sheet1.csv"
    dataframe = read_csv_to_dataframe(csv_filename)

    # Load the BERT model and tokenizer
    bert_model_name = 'bert-base-uncased'
    bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
    bert_model = BertModel.from_pretrained(bert_model_name)

    # Load the BART model and tokenizer
    bart_model_name = 'facebook/bart-base'
    bart_tokenizer = BartTokenizer.from_pretrained(bart_model_name)
    bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name)

    csv_filename = "summ3.csv"

    for index in range(len(dataframe)):
        text = dataframe.loc[index, 'article']
        summ = summarization(text)
        append_sentence_to_csv(summ, csv_filename)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
import csv
import pandas as pd
from rouge import Rouge

def read_csv_to_dataframe(filename):
    df = pd.read_csv(filename)
    return df

if __name__ == "__main__":
  csv_1 = "/content/English1 - Sheet1.csv"
  act_df = read_csv_to_dataframe(csv_1)
  act_sum = pd.DataFrame(act_df)
  act_su = act_sum['highlights'].tolist()
  print(type(act_su))

  csv_2 = "/content/summ3 (1) - summ3 (1).csv.csv"
  gen_df = read_csv_to_dataframe(csv_2)
  gen_sum = pd.DataFrame(gen_df)
  gen_su = gen_sum['summary'].tolist()
  print(type(gen_su))

  rouge = Rouge()
  scores = rouge.get_scores(gen_su,act_su)

  rouge_1_f_score1 = scores[0]["rouge-1"]["f"]
  rouge_2_f_score2 = scores[0]["rouge-2"]["f"]
  rouge_l_f_score3 = scores[0]["rouge-l"]["f"]

  print("ROUGE-1 F-score:", rouge_1_f_score1)
  print("ROUGE-2 F-score:", rouge_2_f_score2)
  print("ROUGE-L F-score:", rouge_l_f_score3)


<class 'list'>
<class 'list'>
ROUGE-1 F-score: 0.2173913018899811
ROUGE-2 F-score: 0.06876790659715443
ROUGE-L F-score: 0.2173913018899811


In [ ]:
rouge_1_f1 = scores[0]['rouge-1']['f']
rouge_1_recall = scores[0]['rouge-1']['r']
rouge_1_precision = scores[0]['rouge-1']['p']

print("ROUGE-1 Metric:")
print(rouge_1_f1)
print(rouge_1_recall)
print(rouge_1_precision)

rouge_2_f1 = scores[0]['rouge-2']['f']
rouge_2_recall = scores[0]['rouge-2']['r']
rouge_2_precision = scores[0]['rouge-2']['p']

print("ROUGE-2 Metric:")
print(rouge_2_f1)
print(rouge_2_recall)
print(rouge_2_precision)

rouge_l_f1 = scores[0]['rouge-l']['f']
rouge_l_recall = scores[0]['rouge-l']['r']
rouge_l_precision = scores[0]['rouge-l']['p']

print("ROUGE-l Metric:")
print(rouge_l_f1)
print(rouge_l_recall)
print(rouge_l_precision)

ROUGE-1 Metric:
0.2173913018899811
0.7575757575757576
0.12690355329949238
ROUGE-2 Metric:
0.06876790659715443
0.36363636363636365
0.0379746835443038
ROUGE-l Metric:
0.2173913018899811
0.7575757575757576
0.12690355329949238


In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BartForConditionalGeneration, BartTokenizer
import pandas as pd
import csv

def read_csv_to_dataframe(filename):
    df = pd.read_csv(filename)
    return df

def append_sentence_to_csv(sentence, filename):
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([sentence])

def summarization(inp):
    # Define the input text
    input_text = inp

    # Tokenize the input text with BERT tokenizer
    tokens = bert_tokenizer(input_text, truncation=True, max_length=300, padding='do_not_pad', return_tensors='pt')

    # Extract BERT features
    with torch.no_grad():
        bert_outputs = bert_model(**tokens)

    # Generate the summary using BART
    bart_inputs = bart_tokenizer.prepare_seq2seq_batch([input_text], truncation=True, max_length=1024, padding='do_not_pad', return_tensors='pt')

    summary_ids = bart_model.generate(
        input_ids=bart_inputs.input_ids.to(bart_model.device),
        attention_mask=bart_inputs.attention_mask.to(bart_model.device),
        max_length=50,
        min_length=100,
        num_beams=2,
        length_penalty=1.0,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # Decode the summary tokens
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

if __name__ == "__main__":
    csv_filename = "/content/English1 - Sheet1.csv"
    dataframe = read_csv_to_dataframe(csv_filename)

    # Load the BERT model and tokenizer
    bert_model_name = 'bert-base-uncased'
    bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
    bert_model = BertModel.from_pretrained(bert_model_name)

    # Load the BART model and tokenizer
    bart_model_name = 'facebook/bart-base'
    bart_tokenizer = BartTokenizer.from_pretrained(bart_model_name)
    bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name)

    csv_filename = "summ4.csv"

    for index in range(len(dataframe)):
        text = dataframe.loc[index, 'article']
        summ = summarization(text)
        append_sentence_to_csv(summ, csv_filename)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1165: UserWarning: Unfeasible length constraints: `min_

In [ ]:
import csv
import pandas as pd
from rouge import Rouge

def read_csv_to_dataframe(filename):
    df = pd.read_csv(filename)
    return df

if __name__ == "__main__":
  csv_1 = "/content/English1 - Sheet1.csv"
  act_df = read_csv_to_dataframe(csv_1)
  act_sum = pd.DataFrame(act_df)
  act_su = act_sum['highlights'].tolist()
  print(type(act_su))

  csv_2 = "/content/summ4 - summ4.csv.csv"
  gen_df = read_csv_to_dataframe(csv_2)
  gen_sum = pd.DataFrame(gen_df)
  gen_su = gen_sum['summary'].tolist()
  print(type(gen_su))

  rouge = Rouge()
  scores = rouge.get_scores(gen_su,act_su)

  rouge_1_f_score11 = scores[0]["rouge-1"]["f"]
  rouge_2_f_score21 = scores[0]["rouge-2"]["f"]
  rouge_l_f_score31 = scores[0]["rouge-l"]["f"]

  print("ROUGE-1 F-score:", rouge_1_f_score11)
  print("ROUGE-2 F-score:", rouge_2_f_score21)
  print("ROUGE-L F-score:", rouge_l_f_score31)


<class 'list'>
<class 'list'>
ROUGE-1 F-score: 0.31999999507200005
ROUGE-2 F-score: 0.13157894245498633
ROUGE-L F-score: 0.2933333284053334


In [ ]:
rouge_1_f1 = scores[0]['rouge-1']['f']
rouge_1_recall = scores[0]['rouge-1']['r']
rouge_1_precision = scores[0]['rouge-1']['p']

print("ROUGE-1 Metric:")
print(rouge_1_f1)
print(rouge_1_recall)
print(rouge_1_precision)

rouge_2_f1 = scores[0]['rouge-2']['f']
rouge_2_recall = scores[0]['rouge-2']['r']
rouge_2_precision = scores[0]['rouge-2']['p']

print("ROUGE-2 Metric:")
print(rouge_2_f1)
print(rouge_2_recall)
print(rouge_2_precision)

rouge_l_f1 = scores[0]['rouge-l']['f']
rouge_l_recall = scores[0]['rouge-l']['r']
rouge_l_precision = scores[0]['rouge-l']['p']

print("ROUGE-l Metric:")
print(rouge_l_f1)
print(rouge_l_recall)
print(rouge_l_precision)

ROUGE-1 Metric:
0.31999999507200005
0.36363636363636365
0.2857142857142857
ROUGE-2 Metric:
0.13157894245498633
0.15151515151515152
0.11627906976744186
ROUGE-l Metric:
0.2933333284053334
0.3333333333333333
0.2619047619047619
